In [1]:
import os
import cv2
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.preprocessing import image


In [2]:
json_path = 'E:/AInotes/자세교정/모델학습/label_data.json'
exercise_name = 'pushup'
model = tf.keras.models.load_model(r'E:\AImodel\models\Multilabel\pushup.h5')
dir_path = r'E:/OneDrive - 가천대학교/4학년 1학기/졸프/운동영상/pushup/01'

print(model.input.shape)
print(model.input.dtype)
print(model.output.shape)
print(model.output.dtype)


(None, 128, 128, 3)
<dtype: 'float32'>
(None, 5)
<dtype: 'float32'>


In [3]:
def preprocess_image(image_path, target_size=(128, 128)): #이미지 파일을 불러와 전처리하는 함수
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델 예측을 위해 차원 확장
    img_array /= 255.0  # 이미지를 0과 1 사이로 스케일링
    return img_array


def load_and_preprocess_from_directory(directory_path, target_size=(128, 128)): # 지정된 디렉토리 내의 모든 이미지를 불러와 전처리하는 함수
    processed_images = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = preprocess_image(file_path, target_size=target_size)
            processed_images.append(img)
    return np.vstack(processed_images)  # 전처리된 이미지들을 하나의 numpy 배열로 합침


def apply_sliding_window_to_predictions(predictions, window_size=5, method='mean'): # 2D 예측 결과 배열에 대해 슬라이딩 윈도우를 적용하는 함수
    num_classes = predictions.shape[1]  # 클래스 수
    smoothed_predictions = np.zeros_like(predictions)
    
    for class_idx in range(num_classes):
        class_predictions = predictions[:, class_idx]  # 현재 클래스에 대한 모든 예측값
        # 각 예측값에 대해 슬라이딩 윈도우 적용
        for i in range(len(class_predictions)):
            start_idx = max(0, i - window_size // 2)
            end_idx = min(len(class_predictions), i + window_size // 2 + 1)
            window = class_predictions[start_idx:end_idx]
            if method == 'mean':
                smoothed_value = np.mean(window)
            elif method == 'median':
                smoothed_value = np.median(window)
            else:
                raise ValueError("Method should be 'mean' or 'median'")
            smoothed_predictions[i, class_idx] = smoothed_value
    
    return smoothed_predictions


def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)


def get_prefix_to_label(file_path, exercise_name):
    data = load_json_file(file_path)
    result = (data.get(exercise_name))
    prefix = data.get(exercise_name)['prefix']
    label = data.get(exercise_name)['last_path']
    prefix_to_label = dict(zip(prefix, label))
    return prefix_to_label
    

def print_predictions(predictions, exercise_name, json_path, threshold=0.5):
    # JSON 파일 로드 및 라벨 길이 구하기
    data = load_json_file(json_path)
    label_length = len(data[exercise_name]['last_path'][0])
    
    # 각 라벨마다 T, F의 개수를 저장할 딕셔너리 동적으로 초기화
    label_counts = {i+1: {'T': 0, 'F': 0} for i in range(label_length)}
    
    for i, prediction in enumerate(predictions):
        labels = (prediction > threshold).astype(int)
        label_str = ','.join(['T' if label == 1 else 'F' for label in labels])
        print(f"Image {i+1}: {label_str}")
        
        # 각 라벨마다 T, F의 개수를 계산
        for label_index, label in enumerate(labels, start=1):
            if label == 1:
                label_counts[label_index]['T'] += 1
            else:
                label_counts[label_index]['F'] += 1
    
    # 각 라벨에 대한 총합 출력
    for label_index in label_counts:
        print(f"Label {label_index}: T = {label_counts[label_index]['T']}, F = {label_counts[label_index]['F']}")


preprocessed_images = load_and_preprocess_from_directory(dir_path)
predictions = model.predict(preprocessed_images, verbose=0)
# 예측 결과 출력
print(predictions)
smoothed_predictions = apply_sliding_window_to_predictions(predictions, window_size=5, method='mean')




[[0.99997616 0.99691904 0.00595089 0.99994135 0.9999459 ]
 [0.9999727  0.9992353  0.00170062 0.9987942  0.9988097 ]
 [0.9999572  0.9997727  0.00280166 0.9984378  0.15122992]
 [0.99975044 0.92921764 0.34324864 0.8754566  0.996958  ]
 [0.9997228  0.9793189  0.01159788 0.9782055  0.98910743]
 [0.9996934  0.89891285 0.08897965 0.97646517 0.99921536]
 [0.9997044  0.99767274 0.00489086 0.99568605 0.99442357]
 [0.99964154 0.9670001  0.0194758  0.91630757 0.9988041 ]
 [0.99972636 0.9924912  0.01615317 0.99437666 0.97630113]
 [0.9994956  0.9830474  0.02793821 0.97027045 0.9989348 ]
 [0.9996158  0.99293983 0.01394386 0.99471766 0.99662113]
 [0.99985695 0.9923194  0.00482072 0.864315   0.9989023 ]]


In [4]:
# 모델의 출력층에 접근하여 클래스의 개수 얻기
output_layer = model.layers[-1]  # 모델의 마지막 층 (출력층)을 가져옴
num_classes = output_layer.units  # 출력층의 유닛 수를 가져옴, 이는 클래스의 수와 동일

print(f'모델은 총 {num_classes}개의 클래스를 가지고 있습니다.')
class_labels = []
# 클래스 라벨 정의 (예: num_classes 개의 클래스)
for i in range(num_classes):
    class_labels.append(f'class{i+1}')

# 예를 들어, num_classes가 3이라면 class_labels는 ['class1', 'cd lass2', 'class3']이 됩니다.


# predictions 배열에서 각 입력에 대한 모든 클래스의 확률 출력
for i, prediction in enumerate(predictions):
    print(f'Image {i+1}:')
    for j, class_label in enumerate(class_labels):
        print(f'  {class_label}: {prediction[j]:.2f}%')
    print()  # 각 이미지의 예측 결과 사이에 공백 추가


print_predictions(predictions, exercise_name, json_path)



모델은 총 5개의 클래스를 가지고 있습니다.
Image 1:
  class1: 1.00%
  class2: 1.00%
  class3: 0.01%
  class4: 1.00%
  class5: 1.00%

Image 2:
  class1: 1.00%
  class2: 1.00%
  class3: 0.00%
  class4: 1.00%
  class5: 1.00%

Image 3:
  class1: 1.00%
  class2: 1.00%
  class3: 0.00%
  class4: 1.00%
  class5: 0.15%

Image 4:
  class1: 1.00%
  class2: 0.93%
  class3: 0.34%
  class4: 0.88%
  class5: 1.00%

Image 5:
  class1: 1.00%
  class2: 0.98%
  class3: 0.01%
  class4: 0.98%
  class5: 0.99%

Image 6:
  class1: 1.00%
  class2: 0.90%
  class3: 0.09%
  class4: 0.98%
  class5: 1.00%

Image 7:
  class1: 1.00%
  class2: 1.00%
  class3: 0.00%
  class4: 1.00%
  class5: 0.99%

Image 8:
  class1: 1.00%
  class2: 0.97%
  class3: 0.02%
  class4: 0.92%
  class5: 1.00%

Image 9:
  class1: 1.00%
  class2: 0.99%
  class3: 0.02%
  class4: 0.99%
  class5: 0.98%

Image 10:
  class1: 1.00%
  class2: 0.98%
  class3: 0.03%
  class4: 0.97%
  class5: 1.00%

Image 11:
  class1: 1.00%
  class2: 0.99%
  class3: 0.01%
  class4: 0.99%


In [5]:
print(f'모델은 총 {num_classes}개의 클래스를 가지고 있습니다.')
class_labels = []
# 클래스 라벨 정의 (예: num_classes 개의 클래스)
for i in range(num_classes):
    class_labels.append(f'class{i+1}')

# 예를 들어, num_classes가 3이라면 class_labels는 ['class1', 'cd lass2', 'class3']이 됩니다.


# predictions 배열에서 각 입력에 대한 모든 클래스의 확률 출력
for i, prediction in enumerate(smoothed_predictions):
    print(f'Image {i+1}:')
    for j, class_label in enumerate(class_labels):
        print(f'  {class_label}: {prediction[j]:.2f}%')
    print()  # 각 이미지의 예측 결과 사이에 공백 추가
# 부드러운 예측 결과 출력


print_predictions(smoothed_predictions, exercise_name, json_path)

모델은 총 5개의 클래스를 가지고 있습니다.
Image 1:
  class1: 1.00%
  class2: 1.00%
  class3: 0.00%
  class4: 1.00%
  class5: 0.72%

Image 2:
  class1: 1.00%
  class2: 0.98%
  class3: 0.09%
  class4: 0.97%
  class5: 0.79%

Image 3:
  class1: 1.00%
  class2: 0.98%
  class3: 0.07%
  class4: 0.97%
  class5: 0.83%

Image 4:
  class1: 1.00%
  class2: 0.96%
  class3: 0.09%
  class4: 0.97%
  class5: 0.83%

Image 5:
  class1: 1.00%
  class2: 0.96%
  class3: 0.09%
  class4: 0.96%
  class5: 0.83%

Image 6:
  class1: 1.00%
  class2: 0.95%
  class3: 0.09%
  class4: 0.95%
  class5: 1.00%

Image 7:
  class1: 1.00%
  class2: 0.97%
  class3: 0.03%
  class4: 0.97%
  class5: 0.99%

Image 8:
  class1: 1.00%
  class2: 0.97%
  class3: 0.03%
  class4: 0.97%
  class5: 0.99%

Image 9:
  class1: 1.00%
  class2: 0.99%
  class3: 0.02%
  class4: 0.97%
  class5: 0.99%

Image 10:
  class1: 1.00%
  class2: 0.99%
  class3: 0.02%
  class4: 0.95%
  class5: 0.99%

Image 11:
  class1: 1.00%
  class2: 0.99%
  class3: 0.02%
  class4: 0.96%
